<a href="https://colab.research.google.com/github/elsa9421/Interactive-IPython-Demos/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Uncomment the following line of code to install ipywidgets
#!pip install ipywidgets  # installing widgets required for interactive sliders

import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets
import random
from math import pi,cos,sin,sqrt,log
from scipy.stats.distributions import chi2   #chi2.ppf(0.975, df=2)
from scipy.stats import multivariate_normal

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.special import expit

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
def Cov_matrix(L,theta=pi/4):
  '''
  The function returns a 2x2 covariance matrix parmeterized by L and theta
  The covariance matrix is generated by R*[[L,0],[0,1]]*R.T
  where R is the rotation matrix


  '''

  Rot_matrix=np.array([[cos(theta),-sin(theta)],[sin(theta),cos(theta)]])
 
  M=np.array([[L,0],[0,1]])
 # print("Dot product",np.dot(Rot_matrix,M))
  cov=np.dot(np.dot(Rot_matrix,M),Rot_matrix.T)
  

  return cov



def plot_ellipse(m,S):
  '''
  Plot an Elliptical contour of a bivariate Gaussian density

  Input:
  -m:mean
  -S:covariance matrix

  Output:
  -z : ellipse boundary points for plotting using
      >> plot(z[0,:],z[1,:])
  '''

  d=len(m)
  if d!=2:
    sys.exit("Plot ellipse assumes 2D data")
  
  #beta= theoretical mass enclosed
  beta=.9
  r=sqrt(chi2.ppf(beta, df=2))


  #Plot circle according to the given covariance matrix  -> https://cookierobotics.com/007/
  N=500
  t=np.linspace(0,2*pi,N+1)[:-1]
  u=r*np.array([np.cos(t),np.sin(t)])
  eigVal, eigVec = np.linalg.eig(S)

  z=np.dot(np.dot(eigVec,np.sqrt(np.diag(eigVal))),u) + m.reshape(-1,1)

  return z


def Mahalanobis_dist(x,mu,cov):
  '''
  Calculates Mahalanobis Distance
  -x: a 2D point ie of shape(2,)
  -mu: mean of shape(2,)
  -cov: Covariance matrix of shape(2,2)

  '''
  
  Cov_inv=np.linalg.inv(cov)
  x_mu=x-mu
  m_d=np.dot(x_mu.T, np.dot(Cov_inv,x_mu))


  return m_d

  




In [ ]:
def LDA(X,y):
  '''
  Estimating and plotting LDA decision boundary by calculting mean, covariance and probability of each class
  and plugging into Bayes' classifier formula

  -X : 2D data, of shape(N,2)
  -y : labels for the data, of shape(N,)
  '''
  # Calculating estimated parameters using MLE:
  sample={}

  sample[0]=X[y==0,:]     #shape (300,2) if N=600  and prob of each class=1/2
  sample[1]=X[y==1,:]     #shape (300,2) if N=600
  

  p,mu={},{}
  p[0]=len(sample[0])/len(y)   
  p[1]=len(sample[1])/len(y)
  #print("Probabitlites")
  #print(p[0],p[1])

  mu[0]=np.mean(sample[0],axis=0)
  mu[1]=np.mean(sample[1],axis=0)
 # print("Mean")
 # print(mu[0],mu[1])

  # Calculating Pooled covariance estimate:
  term1=sample[0].T-mu[0].reshape(-1,1)  #shape(2,N)
  term2=sample[1].T-mu[1].reshape(-1,1)  #shape(2,N)


  x_minus_mu=np.concatenate((term1,term2),axis=1)     #shape(2,2N)
 

  Cov=(1/len(X))*np.dot(x_minus_mu,x_minus_mu.T)
  #To print mean and covariance
  #print(Cov)
  #print("mean",mu[0],mu[1])

  
  # Plot Decision boundary 
 
  y_intercept=(log(p[0]/p[1]))-(1/2)*np.dot(mu[0].T,mu[0])+(1/2)*np.dot(mu[1].T,mu[1])
  temp=np.dot(np.linalg.inv(Cov),mu[1]-mu[0])
  Slope=-temp[0]/temp[1]
 # print("log part=",y_intercept,"temp[1]=",temp[1])
  y_intercept=y_intercept/temp[1]
  x_vec=np.linspace(-15,15)
  
  #print("Slope=",Slope,"Intercept=",y_intercept)
  plt.plot(x_vec,(Slope*x_vec)+y_intercept,color='black')
  plt.title('LDA')



In [ ]:
def LDA_predefined(X,y):
    '''
    Estimating and plotting decision boundary for LDA classifier using python library sklearn.discriminant_analysis 
    -X : 2D data, of shape(N,2)
    -y : labels for the data, of shape(N,)

    '''
    lda = LinearDiscriminantAnalysis(solver="svd", store_covariance=True)

    y_pred = lda.fit(X, y)
     
    #To print mean and covariance
    #print(y_pred.covariance_)
    #print("mean",y_pred.means_)
      

    # Plot Decision boundary 

    x_vec=np.linspace(-15,15)


    plt.plot(x_vec,-(y_pred.coef_[0][0]/y_pred.coef_[0][1])* x_vec - y_pred.intercept_/y_pred.coef_[0][1],color='black')
    plt.title('LDA predefined')



In [ ]:

def plot_MVG(parameter=1,theta=0,LDA_type='predefined'):
  mu={}
  Sigma={}
  sample={}
  c={0:'purple',1:'yellow'}
  sym={0:'.',1:'x'}
  mu[0]=np.array([-5,0])
  Sigma[0]=Cov_matrix(parameter,theta)
 

  mu[1]=np.array([5,0])
  Sigma[1]=Sigma[0]



  #sample some points
  for k in range(len(mu)):
    #sample[k]=np.random.multivariate_normal(mu[k],Sigma[k],300)
    mn = multivariate_normal(mean=mu[k], cov=Sigma[k])
    sample[k]=mn.rvs(size=300, random_state=42)
  
  
  plt.figure(figsize=(7,7))

  fig=plt.gcf()
  fig.set_facecolor('white')
  plt.axis('equal')
  plt.xlim(-15,15)
  plt.ylim(-15,15)
  

  for k in range(len(mu)):
    plt.plot(sample[k][:,0],sample[k][:,1],sym[k],color=c[k])
    z=plot_ellipse(mu[k],Sigma[k])
    plt.plot(z[0,:],z[1,:],color=c[k])


  # data to be passed to LDA function
  y0=np.zeros((len(sample[0]),))
  y1=np.ones((len(sample[1]),))
  X=np.concatenate((sample[0],sample[1]),axis=0)
  label=np.concatenate((y0,y1)) 
  
  if LDA_type=='predefined':
    LDA_predefined(X=X,y=label)
  else: 
    LDA(X=X,y=label)
    #pass
  

# The interactive plot

lambda_widget=widgets.IntSlider(value=1,
    min=1,
    max=25,
    step=1,
    description='Lambda',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

theta_widget=widgets.FloatSlider(value=45*(pi/180),
    min=0,
    max=90*(pi/180),
    step=pi/180,
    description='theta',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    )

%time interact(plot_MVG,parameter=lambda_widget,theta=theta_widget,continuous_update=False)


interactive(children=(IntSlider(value=1, continuous_update=False, description='Lambda', max=25, min=1), FloatS…

CPU times: user 202 ms, sys: 10.9 ms, total: 212 ms
Wall time: 306 ms


<function __main__.plot_MVG>

## ALTERNATIVE: 
Wihout using predefined python libraries for LDA

In [ ]:
# The interactive plot

lambda_widget=widgets.IntSlider(value=1,
    min=1,
    max=25,
    step=1,
    description='Lambda',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

theta_widget=widgets.FloatSlider(value=45*(pi/180),
    min=0,
    max=90*(pi/180),
    step=pi/180,
    description='theta',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    )

%time interact(plot_MVG,parameter=lambda_widget,theta=theta_widget,LDA_type='not predefined',continuous_update=False)




interactive(children=(IntSlider(value=1, continuous_update=False, description='Lambda', max=25, min=1), FloatS…

CPU times: user 241 ms, sys: 69.2 ms, total: 310 ms
Wall time: 266 ms


<function __main__.plot_MVG>